In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

plt.rcParams.update({
    "figure.dpi": 600,
    "figure.figsize": (6, 2.5),
    "font.size": 8,
    "mathtext.fontset": "stix"
})

In [2]:
# Load vmem simulation data
base_dir = '/Users/susannedahle/CellTypeDependenceElStim/simulation_data/vmem_data_neo' 

vmem_amp_file_paths = [
    os.path.join(base_dir, f'vmem_amp_data_neo_Ez_{i}.npy')
    for i in range(16)
]

vmem_amp_data_list = [
    np.load(path, allow_pickle=True).item()
    for path in vmem_amp_file_paths
]
vmem_amp_data = {}
for data in vmem_amp_data_list:
    vmem_amp_data.update(data)

In [8]:
print(vmem_amp_data['L1_DAC_bNAC219_1'].keys())

dict_keys(['freq', 'soma_amp', 'closest_z_endpoint', 'distant_z_endpoint', 'upper_z_endpoint', 'bottom_z_endpoint', 'total_len', 'relative_z_len', 'symmetry_factor', 'number_of_soma_dendrites', 'total_diam_soma_dendrites', 'soma_diam', 'tot_z_diam_cos', 'tot_z_diam_abs', 'relative_z_diam', 'avg_z_diam'])


In [18]:
# --- Add asymmetry_factor to each cell ---
for cell_name, cell_data in vmem_amp_data.items():
    upper = cell_data.get('upper_z_endpoint')
    bottom = cell_data.get('bottom_z_endpoint')
    
    # Avoid division by zero or missing values
    if upper is not None and bottom is not None and (upper + bottom) != 0:
        asym_factor = np.abs(np.abs(upper) - np.abs(bottom)) / (np.abs(upper) + np.abs(bottom))
    else:
        asym_factor = np.nan  # use NaN for missing or invalid data
    
    cell_data['asymmetry_factor'] = asym_factor

# --- Save back into the same split files (optional but recommended) ---
# We'll rebuild the same split as before
# You can skip this section if you only need one combined file

# Reload each file separately to update its respective cells
for path in vmem_amp_file_paths:
    data = np.load(path, allow_pickle=True).item()
    for cell_name in data.keys():
        if cell_name in vmem_amp_data:
            data[cell_name]['asymmetry_factor'] = vmem_amp_data[cell_name]['asymmetry_factor']
    np.save(path, data)

print("✅ Added 'asymmetry_factor' to all cells and updated .npy files successfully.")

✅ Added 'asymmetry_factor' to all cells and updated .npy files successfully.


In [19]:
# Load vmem simulation data again to check
base_dir = '/Users/susannedahle/CellTypeDependenceElStim/simulation_data/vmem_data_neo' 

vmem_amp_file_paths = [
    os.path.join(base_dir, f'vmem_amp_data_neo_Ez_{i}.npy')
    for i in range(16)
]

vmem_amp_data_list = [
    np.load(path, allow_pickle=True).item()
    for path in vmem_amp_file_paths
]
vmem_amp_data_new = {}
for data in vmem_amp_data_list:
    vmem_amp_data_new.update(data)

In [20]:
vmem_amp_data_new['L1_DAC_bNAC219_1'].keys()

dict_keys(['freq', 'soma_amp', 'closest_z_endpoint', 'distant_z_endpoint', 'upper_z_endpoint', 'bottom_z_endpoint', 'total_len', 'relative_z_len', 'symmetry_factor', 'number_of_soma_dendrites', 'total_diam_soma_dendrites', 'soma_diam', 'tot_z_diam_cos', 'tot_z_diam_abs', 'relative_z_diam', 'avg_z_diam', 'asymmetry_factor'])

In [21]:
print('Example L1_DAC_bNAC219_1:')
print('l_1 = ', vmem_amp_data_new['L1_DAC_bNAC219_1']['upper_z_endpoint'])
print('l_2 = ', vmem_amp_data_new['L1_DAC_bNAC219_1']['bottom_z_endpoint'])
print('symmetry factor = ', vmem_amp_data_new['L1_DAC_bNAC219_1']['symmetry_factor'])
print('Asymmetry factor = ', vmem_amp_data_new['L1_DAC_bNAC219_1']['asymmetry_factor'])

Example L1_DAC_bNAC219_1:
l_1 =  226.20668601989746
l_2 =  -228.89045882882647
symmetry factor =  0.9882748594123968
Asymmetry factor =  0.005897142707456676
